In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.master('local').appName('app').getOrCreate()

In [3]:
sc = spark.sparkContext

In [5]:
df = spark.read.json("file:///home/alan/spark/guided-capstone/part-00000-092ec1db-39ab-4079-9580-f7c7b516a283-c000.txt")

In [6]:
df1 = df.select('bid_pr')

In [8]:
df.schema

StructType(List(StructField(ask_pr,DoubleType,true),StructField(ask_size,LongType,true),StructField(bid_pr,DoubleType,true),StructField(bid_size,LongType,true),StructField(event_seq_nb,LongType,true),StructField(event_tm,StringType,true),StructField(event_type,StringType,true),StructField(exchange,StringType,true),StructField(execution_id,StringType,true),StructField(file_tm,StringType,true),StructField(price,DoubleType,true),StructField(size,LongType,true),StructField(symbol,StringType,true),StructField(trade_dt,StringType,true)))

In [7]:
x = df1.take(5)
print(x[0])
df1.printSchema()

Row(bid_pr=78.13370587077013)
root
 |-- bid_pr: double (nullable = true)



In [22]:
df2 = df1.withColumn('bid_pr', F.col('bid_pr').cast(T.DecimalType(10,2)))

In [44]:
df2.printSchema()

root
 |-- bid_pr: decimal(10,2) (nullable = true)



In [30]:
df3 = df.select('trade_dt').withColumn('trade_dt', F.col('trade_dt').cast(T.DateType()) )

In [47]:
x = ['a', 'b', 'c']
x



['a', 'b', 'c']

In [43]:
type(df3.withColumn('next_day', F.col('trade_dt') + 1).take(5)[0]['next_day'])


datetime.date

In [40]:
a = float(10.99)

In [33]:
y.collect()

[[1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 

In [40]:
out = raw.collect()
print(out[0])

{"trade_dt":"2020-08-06","file_tm":"2020-08-06 09:30:00.000","event_type":"Q","symbol":"SYMA","event_tm":"2020-08-06 09:38:08.093","event_seq_nb":1,"exchange":"NASDAQ","bid_pr":78.13370587077013,"bid_size":100,"ask_pr":79.8251633824899,"ask_size":100}


In [8]:
import json

def parse_json(line : str):
    record = json.loads(line)
    record_type = record['event_type']
    record_keys = set(record.keys())
    
    def translate(s):
                      
        translations = {'event_type' : 'rec_type', 'file_tm' : 'arrival_tm', 'price' : 'trade_pr'}
        return s if s not in translations else translations[s]
    
    quote_columns = {'trade_dt', 'file_tm', 'event_type', 'symbol', 'event_tm', 'event_seq_nb',
                    'exchange', 'bid_pr', 'bid_size', 'ask_pr', 'ask_size'}
    
    trade_columns = {'trade_dt', 'file_tm', 'event_type', 'symbol', 'event_tm', 'event_seq_nb',
                    'exchange', 'price', 'size', 'execution_id'}
    
    is_valid_record = (record_type == 'Q' and record_keys == quote_columns) or \
                      (record_type == 'T' and record_keys == trade_columns)
   
    if is_valid_record:
       translated_record =  {translate(k):v for k,v in record.items()}  
       translated_record['partition'] = record_type                
       return get_common_event(translated_record)                      
    else:
       translated_record['partition'] = 'B' 
       translated_record['line'] = line              
       return get_common_event(translated_record)
    
    


In [20]:
for k in d.keys():
    print(k, d[k])

trade_dt 2020-08-06
file_tm 2020-08-06 09:30:00.000
event_type Q
symbol SYMA
event_tm 2020-08-06 09:38:08.093
event_seq_nb 1
exchange NASDAQ
bid_pr 78.13370587077013
bid_size 100
ask_pr 79.8251633824899
ask_size 100


In [33]:
def get_common_event(record : dict ):
    
    common_data = ('trade_dt', 'rec_type', 'symbol', 'exchange', 'event_tm', 'event_seq_nb','arrival_tm', 
                   'trade_pr',  'bid_pr', 'bid_size', 'ask_pr', 'ask_size', 'partition')
    
    return [record.get(k,"") for k in common_data] 
    
    